### Import Modules

In [257]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [285]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [286]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 10, # max 10
        'exclude' : [], # put LECs here
        'include_log' : True, # alphaS
        'include_log2' : False
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'xpt-taylor', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : False,
    
    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

### Do fit

In [287]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Using default prior.
Making fits...
Done!
100% complete Time (s):  1.36100006104
Compiling results...


Fit type: xpt-taylor (F^2 = FKFpi, bsN = 1)

Fitting to nnlo 
 with volume corrections O(10) 
 chained: False 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections):

F_K / F_pi = 1.1793(56) [1.1932(19)]   (delta_su2 = -0.00193(24))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
   A_loga 0              -1.5 (2.3)            1               0.0 (3.5)
          1               0.0 (2.7)        A_a 0              -2.8 (1.5)
      L_5 0           -0.00045 (46)            1               0.0 (2.7)
          1             0.0010 (50)        A_p 0               1.48 (36)
      A_k 0               2.7 (2.0)            1               0.0 (3.5)
---
boot0 fit results:
Least Square Fit:
  chi2/dof [dof] = 1 [16]    Q = 0.42    logGBF = 66.54

Settings:
  svdcut/n = 1e-12/0

In [289]:
#print bootstrapper.fits[0]

In [275]:
for abbr in bootstrapper.abbrs:
    print abbr, bootstrapper.fit_data[abbr]['alpha_s']

a09m220 0.43356000(43)
a09m310 0.43356000(43)
a09m350 0.43356000(43)
a09m400 0.43356000(43)
a12m130 0.53796000(54)
a12m220 0.53796000(54)
a12m220L 0.53796000(54)
a12m220S 0.53796000(54)
a12m310 0.53796000(54)
a12m350 0.53796000(54)
a12m400 0.53796000(54)
a15m135XL 0.58801000(59)
a15m220 0.58801000(59)
a15m310 0.58801000(59)
a15m350 0.58801000(59)
a15m400 0.58801000(59)


In [274]:
print [bootstrapper.fit_data[abbr]['alpha_s'] for abbr in bootstrapper.abbrs]

[0.43356000(43), 0.43356000(43), 0.43356000(43), 0.43356000(43), 0.53796000(54), 0.53796000(54), 0.53796000(54), 0.53796000(54), 0.53796000(54), 0.53796000(54), 0.53796000(54), 0.58801000(59), 0.58801000(59), 0.58801000(59), 0.58801000(59), 0.58801000(59)]


In [ ]:
def test(self):
    p
    

In [248]:
def test(self):
    temp = bootstrapper.fits[0].p
    print gv.corr(temp['mpi'][0], temp['mk'][0])
    
test(bootstrapper)

0.442983188623


In [235]:
bootstrapper.extrapolate_to_phys_point(include_su2_isospin_corrrection=False)

1.1804(57)

In [190]:
def test(self):
    return gv.corr(self.fit_data['a09m220']['mpi'], self.fit_data['a09m220']['mk'])
test(bootstrapper)

0.46065693199088703

In [185]:
def test(self):
    for abbr in self.abbrs:
        print abbr, self.fit_data[abbr]['FK/Fpi'], self.extrapolate_to_ensemble(abbr)

test(bootstrapper)

a09m220 1.1439(18) 1.1447(12)
a09m310 1.1108(16) 1.11160(62)
a09m350 1.0997(10) 1.09822(64)
a09m400 1.08107(94) 1.08085(61)
a12m130 1.1646(26) 1.1628(12)
a12m220 1.1314(23) 1.13444(83)
a12m220L 1.1279(29) 1.13362(82)
a12m220S 1.1389(27) 1.13783(95)
a12m310 1.1042(21) 1.10467(58)
a12m350 1.0888(27) 1.09154(47)
a12m400 1.0745(17) 1.07609(55)
a15m135XL 1.1584(28) 1.1559(15)
a15m220 1.1266(15) 1.12685(97)
a15m310 1.1016(13) 1.10095(56)
a15m350 1.08868(87) 1.08793(55)
a15m400 1.07151(89) 1.07239(61)


### Or make all fits

In [44]:
p_dict = {
    'order' : {
        'vol' : 5, # max 10
        'exclude' : [],
        'include_log2' : True
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : True,

    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400', u'a15m130', u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],

    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,

    'save_prior' : False,
    'save_results' : True,
}

In [45]:
for fit_type in ['xpt', 'xpt-taylor', 'ma', 'ma-taylor']:#, 'ma-old']:
    for order_fit in ['nlo', 'nnlo']:#, 'nnnlo']:
        
        print fit_type, order_fit
        
        # Specify fit parameters
        p_dict['order']['fit'] = order_fit
        p_dict['fit_type'] = fit_type
        
        # Do fit
        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
            )
        else:
            print bootstrapper

        if p_dict['save_prior']:
            print 'Saving prior...'
            data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])

        if p_dict['save_results']:
            data_loader.save_fit_info(bootstrapper.get_fit_info())


        t1 = time.time()
        
        #print test(bootstrapper)

        print "\nTotal time (s): ", t1 - t0, "\n"

xpt nlo
Making fits...
100% complete Time (s):  0.463999986649
Compiling results...


../fitter\special_functions.py:10: RuntimeWarning: invalid value encountered in double_scalars
  ysdev = np.abs(ss.kvp(n, gv.mean(g), 1)) * gv.sdev(g)


Done.
Saving...
Done.

Total time (s):  11.7779998779 

xpt nnlo
Making fits...
100% complete Time (s):  0.979000091553
Compiling results...
Done.
Saving...
Done.

Total time (s):  12.881000042 

xpt-taylor nlo
Making fits...
100% complete Time (s):  0.468999862671
Compiling results...
Done.
Saving...
Done.

Total time (s):  8.63800001144 

xpt-taylor nnlo
Making fits...
100% complete Time (s):  0.984000205994
Compiling results...
Done.
Saving...
Done.

Total time (s):  9.89199995995 

ma nlo
Making fits...
100% complete Time (s):  0.991000175476
Compiling results...
Done.
Saving...
Done.

Total time (s):  54.1529998779 

ma nnlo
Making fits...
100% complete Time (s):  2.49399995804
Compiling results...
Done.
Saving...
Done.

Total time (s):  56.1860001087 

ma-taylor nlo
Making fits...
100% complete Time (s):  1.11199998856
Compiling results...
Done.
Saving...
Done.

Total time (s):  49.6940000057 

ma-taylor nnlo
Making fits...
100% complete Time (s):  2.46700000763
Compiling results

In [38]:
#bootstrapper.fits[0].show_plots(view='std')